In [1]:
import ee
import geemap
import json
import os
#from geemap import geojson_to_ee
from ipyleaflet import GeoJSON
import segment_testing_my_gee_functions as mgf

In [2]:
ee.Authenticate()
#Initializing the ee project

ee.Initialize(project = 'ee-jdawsey')

In [3]:
shp_folder_path = 'E:/!!Research/!!!Data/example_data/example_data_py/example_data_shp/'
#shp_file = 'example_data.shp'
shp_file = 'ex_data_med.shp'
example_data = geemap.shp_to_ee(f'{shp_folder_path}{shp_file}')

In [4]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [5]:
# checking that the imported layer loadedn
Map.addLayer(example_data)

In [12]:
shp = example_data
shp_geom = shp.geometry()
exp_file_dir_naip = 'E:/!!Research/!!!Data/example_data/example_data_py/naip/ex_naip.tif'

geemap.ee_export_image(
    naip, filename = exp_file_dir_naip, scale=1, region=shp_geom, file_per_band=False
)

Generating URL ...
Please wait ...
Data downloaded to E:\!!Research\!!!Data\example_data\example_data_py\classified\ex_naip.tif


Testing pixel-based classification

In [32]:
naip = mgf.naip_func(example_data)

vis_param_naip = {'bands' : ['R', 'G', 'B'], 
               'min' : 0, 
               'max' : 256,
               'gamma' : 1}

### checking that image was smoothed. Rename as needed
Map.addLayer(naip, vis_param_naip, f'naip')

savi = mgf.naip_savi_endvi(naip)
vis_param_savi = {'bands' : ['savi'], 
               'min' : 0, 
               'max' : 1,
               'gamma' : 1}

"""
endvi = mgf.naip_endvi(savi)
vis_param_savi = {'bands' : ['endvi'], 
               'min' : 0, 
               'max' : 1,
               'gamma' : 1}
"""
#Map.addLayer(savi, vis_param_savi, f'savi')

gauss_three = gauss_three_func(savi)
vis_param_smooth = {'bands' : ['R', 'G', 'B'], 
               'min' : 0, 
               'max' : 256,
               'gamma' : 1}
#Map.addLayer(gauss_three, vis_param_smooth, f'smoothed three')

grayscale = naip.expression(
      '(0.3 * R) + (0.59 * G) + (0.11 * B)', {
      'R': naip.select(['R']),
      'G': naip.select(['G']),
      'B': naip.select(['B'])
})

vis_param_gray = {#'bands' : ['R', 'G', 'B'], 
               'min' : 0, 
               'max' : 256,
               'gamma' : 1}

### checking that image was smoothed. Rename as needed
#Map.addLayer(grayscale, vis_param_gray, f'gray')

# if wanting to use a grayscale image
int_gray = grayscale.int()
glcmTexture = int_gray.glcmTexture(5)

# if not wanting to use a grayscaled image
#int_naip = naip.int()
#glcmTexture = int_naip.glcmTexture(10)

imp_glcm_bands = glcmTexture.select(['constant_savg', 'constant_contrast'])
contrastVis = {
   'bands' : ['constant_savg', 'constant_contrast']
}

#Map.addLayer(imp_glcm_bands, contrastVis, 'contrast')

glcm_segment = imp_glcm_bands.addBands(gauss, ['R', 'G', 'B', 'N', 'savi']) #may need to include 'savi' and 'endvi'
contrastVis_rgb = {
   'bands' : ['R', 'G', 'B']
}

#Map.addLayer(glcm_segment, contrastVis_rgb, 'contrast_rgb')

standardized = mgf.stdrd_func(glcm_segment, example_data)
vis_param_std = {'bands' : ['N', 'R', 'constant_savg'], 
               'min' : -1.5, 
               'max' : 2,
               'gamma' : 1}
#Map.addLayer(standardized, vis_param_std, 'standardized')

# point-based algorithm
max_clusters = 10 # tested with 10

pb_classed = mgf.pb_class(example_data, standardized, max_clusters)
obj_palette = ['#000000','#a6cee3', '#1f78b4', '#b2df8a', '#33a02c', '#fb9a99', '#e31a1c', 
               '#fdbf6f', '#ff7f00', '#cab2d6', '#6a3d9a', '#ffff99', '#b15928']
pb_vis = {'min' : 0, 'max' : 12, 'palette' : obj_palette}
Map.addLayer(pb_classed, pb_vis, f'pixel-based classification with poly_1 and {max_clusters} clusters')

In [ ]:
shp = example_data
shp_geom = shp.geometry()
exp_file_dir_pb = 'E:/!!Research/!!!Data/example_data/example_data_py/classified/ex_pb_classed_10_no_savi.tif'

geemap.ee_export_image(
    pb_classed, filename = exp_file_dir_pb, scale=2, region=shp_geom, file_per_band=False
)